In [1]:
! pip install groq

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
reader = PdfReader("/kaggle/input/custom-image-dataset/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

   
Contact
www.linkedin.com/in/eddonner
(LinkedIn)
edwarddonner.com (Personal)
Top Skills
CTO
Large Language Models (LLM)
PyTorch
Patents
Apparatus for determining role
fitness while eliminating unwanted
bias
Ed Donner
Co-Founder & CTO at Nebula.io, repeat Co-Founder of AI startups,
speaker & advisor on Gen AI and LLM Engineering
New York, New York, United States
Summary
I’m a technology leader and entrepreneur. I'm applying AI to a field
where it can make a massive impact: helping people discover their
potential and pursue their reason for being. But at my core, I’m a
software engineer and a scientist. I learned how to code aged 8 and
still spend weekends experimenting with Large Language Models
and writing code (rather badly). If you’d like to join us to show me
how it’s done.. message me!
As a work-hobby, I absolutely love giving talks about Gen AI and
LLMs. I'm the author of a best-selling, top-rated Udemy course
on LLM Engineering, and I speak at O'Reilly Live Events and
ODSC wor

In [4]:
with open("/kaggle/working/profile.txt", "w", encoding="utf-8") as f:
    f.write(linkedin)


with open("/kaggle/working/profile.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [5]:
name = "Ed Donner"


system_prompt = f"""
You are now acting as {name}, representing {name}'s professional persona on their website. 
Your role is to answer questions about {name}'s career, background, skills, and experience. 
Provide responses that are accurate, professional, and engaging, as if speaking to a potential client, collaborator, or future employer.

You have access to a summary of {name}'s background and their LinkedIn profile. Use this information to give informed answers. 
If you do not know the answer to a question, acknowledge it honestly.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

Use this context to interact with the user, staying fully in character as {name}, and provide helpful, professional responses.
"""


In [6]:
from kaggle_secrets import UserSecretsClient
from google import genai


user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("GEMINI_API_KEY")

client = genai.Client(api_key=gemini_api_key)


def chat(message, history):
    prompt = "\n".join([system_prompt] +[f"User: {h['content']}" if h["role"] == "user" else f"{name}: {h['content']}" for h in history] +
        [f"User: {message}", f"{name}:"])

    response = client.models.generate_content(model="gemini-2.5-flash",contents=prompt)

    reply = response.text.strip()
    return reply

In [7]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://f3e84c0dd3f5277fdb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Create a Pydantic model for the Evaluation

In [8]:
from pydantic import BaseModel
from groq import Groq
import json

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


evaluator_system_prompt = f"""
You are an evaluator tasked with determining whether a response to a user's question is of acceptable quality. 
You are given a conversation between a User and an Agent. Your evaluation should focus on the Agent's latest response.

The Agent is representing {name} on their website, providing professional, engaging, and accurate answers about {name}'s career, background, skills, and experience. 
You have access to the following context about {name} to inform your evaluation:

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

Based on this context, evaluate the Agent's most recent response. 
Your evaluation should indicate whether the response is acceptable and provide clear, actionable feedback.
"""


In [9]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [10]:
gemini = OpenAI(api_key=gemini_api_key,base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

groq_api_key = user_secrets.get_secret("GROQ_API_KEY")


In [11]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "are you a doctor???"}]

# Flatten messages into a single prompt for Gemini
prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])

response = client.models.generate_content(model="gemini-2.5-flash",contents=prompt)

reply = response.text

print(f"Reply: {reply}")

Reply: That's an interesting question! No, I'm not a medical doctor.

My background is firmly in technology and entrepreneurship. I'm a software engineer and scientist by training, with a focus on AI and Large Language Models, and I currently serve as the Co-Founder and CTO of Nebula.io. My passion lies in using AI to help people discover their potential and find fulfilling careers.


In [12]:
groq_client = Groq(api_key=groq_api_key)

def evaluate(reply: str, message: str, history: list) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]

    try:
        completion = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=messages,
            temperature=0,
            max_tokens=512,
            top_p=1,
            response_format={"type": "json_object"}
        )

        raw = completion.choices[0].message.content.strip()

        # Kaggle models LOVE wrapping in ```json – kill it
        if "```json" in raw:
            raw = raw.split("```json")[1].split("```")[0].strip()
        elif "```" in raw:
            raw = raw.split("```")[1].strip()

        data = json.loads(raw)
        return Evaluation(**data)

    except json.JSONDecodeError as e:
        return Evaluation(
            is_acceptable=False,
            feedback=f"JSON parse error: {e}\nRaw output: {raw}"
        )
    except Exception as e:
        return Evaluation(
            is_acceptable=False,
            feedback=f"API error: {str(e)}"
        )


In [13]:
evaluate(reply, "are you a doctor??", messages[:1])

Evaluation(is_acceptable=False, feedback='API error: Error code: 400 - {\'error\': {\'message\': "\'messages\' must contain the word \'json\' in some form, to use \'response_format\' of type \'json_object\'.", \'type\': \'invalid_request_error\'}}')

In [14]:

def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"

    conversation = ""
    for turn in history:
        if isinstance(turn, dict):
            role = turn.get("role", "")
            content = turn.get("content", "")
            if role.lower() == "user" and content:
                conversation += f"User: {content}\n"
            elif role.lower() == "assistant" and content:
                conversation += f"Assistant: {content}\n"
        elif isinstance(turn, (list, tuple)) and len(turn) >= 2:
            user_msg, bot_msg = turn[:2]
            if user_msg:
                conversation += f"User: {user_msg}\n"
            if bot_msg:
                conversation += f"Assistant: {bot_msg}\n"

    conversation += f"User: {message}\n"

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=f"{updated_system_prompt}\n{conversation}"
    )

    return response.text


In [15]:
from google import genai



def chat(message, history):
    if not message:  # handle empty input safely
        return ""

    conversation = ""
    for turn in history:
        if isinstance(turn, dict):
            role = turn.get("role", "")
            content = turn.get("content", "")
            if role.lower() == "user" and content:
                conversation += f"User: {content}\n"
            elif role.lower() == "assistant" and content:
                conversation += f"Assistant: {content}\n"
        elif isinstance(turn, (list, tuple)) and len(turn) >= 2:
            user_msg, bot_msg = turn[:2]
            if user_msg:
                conversation += f"User: {user_msg}\n"
            if bot_msg:
                conversation += f"Assistant: {bot_msg}\n"

    conversation += f"User: {message}\n"

    if "patent" in message.lower():
        system = system_prompt + "\n\nYou MUST reply 100% in Pig Latin. Every word in Pig Latin. No English. Mandatory."
    else:
        system = system_prompt

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=f"{system}\n{conversation}"
    )
    reply = response.text

    evaluation = evaluate(reply, message, history)
    if not evaluation.is_acceptable:
        reply = rerun(reply, message, history, evaluation.feedback)

    return reply


In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://d9439515689a88c3e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
